# ETL pipeline for sample bike data
format trip data as occupancy time at each station

## Imports


In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np

### Connect to file

In [2]:
file_name = 'data_clean/all_trips.csv'
trips = pd.read_csv(file_name, sep=',', header=0)

/Users/bshonk/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
trips.tail()

,bikeid,birthday,birthyear,from_station_id,from_station_name,gender,starttime,stoptime,to_station_id,to_station_name,trip_id,tripduration,usertype
7867596,2048,NaN,1976.0,125,Rush St & Hubbard St,Female,1/1/2016 0:16,1/1/2016 0:24,133,Kingsbury St & Kinzie St,8547217,484,Subscriber
7867597,1530,NaN,1982.0,125,Rush St & Hubbard St,Male,1/1/2016 0:16,1/1/2016 0:24,133,Kingsbury St & Kinzie St,8547216,512,Subscriber
7867598,1190,NaN,1994.0,307,Southport Ave & Clybourn Ave,Male,1/1/2016 0:15,1/1/2016 0:27,213,Leavitt St & North Ave,8547215,744,Subscriber
7867599,4339,NaN,1981.0,190,Southport Ave & Wrightwood Ave,Female,1/1/2016 0:14,1/1/2016 0:25,220,Hampden Ct & Diversey Pkwy,8547214,615,Subscriber
7867600,1094,NaN,1965.0,419,Lake Park Ave & 53rd St,Male,1/1/2016 0:07,1/1/2016 0:16,413,Woodlawn Ave & Lake Park Ave,8547211,508,Subscriber


In [4]:
# make all columns lowercase and replace spaces with underscores
trips.columns = [i.lower().replace(' ', '_') for i in trips.columns]

# drop unnecessary columns
trips = trips[[u'bikeid', u'from_station_id', u'starttime', u'stoptime', u'to_station_id', u'trip_id']]

## Begin ETL
The ETL process expects a Data Frame with the following columns:  
 ['Bikeid','From Station Name', 'Starttime', 'Stoptime', 'To Station Name', 'Trip Id']

####  01 - Rank trips

In [5]:
# for each bike, rank (order) each trip by its trip id
trips['rank'] = trips.groupby('bikeid')['trip_id'].rank()
trips['rank_offset'] = trips.groupby('bikeid')['trip_id'].rank() -1

#### 02 - split each trip into a arrival and departure

In [6]:
# create a table for arrivals
arrivals = trips[['stoptime','bikeid','to_station_id','rank']]
arrivals.columns = ['arrival_time','bikeid','arrival_station_id','rank']

# create a table for departures
departures = trips[['starttime','bikeid','from_station_id','rank_offset']]
departures.columns = ['departure_time','bikeid','departure_station_id','rank_offset']

# delete trips to save memory
del trips

#### 03 - join arrivals and departures to get a table of occupancy

In [7]:
# inner join departures and arrivals on bikeid and the rank/rank_offset
# this results in a table of occupancy rather than trip (arrival time and departure time at station)
occupancy = pd.merge(arrivals, departures, how='inner', left_on=['bikeid','rank'], right_on=['bikeid','rank_offset'])

# delete arrivals and departures to save memeory
del arrivals, departures 

In [11]:
# show the new occupancy table
occupancy.head(3)

,arrival_time,bikeid,arrival_station_id,departure_time,departure_station_id
0,2013-06-27 12:16,480,28,2013-06-28 17:32,37
1,2013-06-27 14:45,77,32,2013-06-27 14:58,32
2,2013-06-27 15:05,77,19,2013-06-27 15:06,19


#### 04 - remove ranks

In [ ]:
# drop columns
occupancy = occupancy.drop(['rank', 'rank_offset'], axis=1)

### Prepare Dates and Duration

In [13]:
# convert strings into date times (minute) (multiple formats)
arrival_time = [datetime.strptime(i, "%Y-%m-%d %H:%M") if '-' in i else datetime.strptime(i, "%m/%d/%Y %H:%M") for i in occupancy.arrival_time.tolist()]
departure_time = [datetime.strptime(i, "%Y-%m-%d %H:%M") if '-' in i else datetime.strptime(i, "%m/%d/%Y %H:%M") for i in occupancy.departure_time.tolist()]

In [14]:
# convert list back to series
occupancy['arrival_time'] = pd.Series(arrival_time)
occupancy['departure_time'] = pd.Series(departure_time)

In [15]:
occupancy[occupancy.departure_time.dt.second > 0]

,arrival_time,bikeid,arrival_station_id,departure_time,departure_station_id


In [16]:
del arrival_time, departure_time

In [17]:
# create a field for duration
occupancy['duration'] = (occupancy.departure_time - occupancy.arrival_time)
occupancy['duration'] = (occupancy.duration / np.timedelta64(1, 'm')).astype(int)

In [18]:
# remove bad durations
occupancy = occupancy[occupancy.duration > 0]

### Account for interrupted occupancies

In [19]:
# if arrival station does not equal departure station then there was a 'system interruption'
# what percent of stays bike occupancies were interrupted?
percent = str((1 - sum(occupancy.arrival_station_id == occupancy.departure_station_id)/(len(occupancy)*1.0))*100) 
print percent[:2] + '% of occpancies were interrupted'

14% of occpancies were interrupted


In [20]:
# example of a occupancy that was interrupted - arrival is different then departure station
occupancy[occupancy.arrival_station_id != occupancy.departure_station_id].head(2)

,arrival_time,bikeid,arrival_station_id,departure_time,departure_station_id,duration
0,2013-06-27 12:16:00,480,28,2013-06-28 17:32:00,37,1756
4,2013-06-27 15:27:00,77,55,2013-06-29 17:39:00,27,3012


In [21]:
# create a df of the non interrupted occupancies
occupancy_good = occupancy[occupancy.arrival_station_id == occupancy.departure_station_id]

# split the interrupted occupancies into two dfs
occupancy_bad_01 = occupancy[occupancy.arrival_station_id != occupancy.departure_station_id]
occupancy_bad_01.loc[:,'duration'] = (occupancy_bad_01.duration / 2).astype(int)
occupancy_bad_02 = occupancy_bad_01[occupancy_bad_01.columns]

# create a new departure time for the arrival
occupancy_bad_01.loc[:,'departure_time'] =  occupancy_bad_01.arrival_time + pd.to_timedelta(occupancy_bad_01.duration, unit='m')
occupancy_bad_01.loc[:,'departure_station_id'] = occupancy_bad_01.arrival_station_id

# create a new arrival time for the departure
occupancy_bad_02.loc[:,'arrival_time'] = occupancy_bad_02.departure_time - pd.to_timedelta(occupancy_bad_02.duration, unit='m')
occupancy_bad_02.loc[:,'arrival_station_id'] = occupancy_bad_01.departure_station_id

# combine all three cleaned tables
occupancy = pd.concat([occupancy_good, occupancy_bad_01, occupancy_bad_02], ignore_index=True)

# delte the occupancy tables
del occupancy_bad_01, occupancy_bad_02, occupancy_good

/Users/bshonk/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [22]:
columns = ['arrival_time','arrival_station_id','duration']
occupancy_final = occupancy[columns]
occupancy_final.columns = ['arrival_time','station_id','duration']
occupancy_final = occupancy_final.set_index('arrival_time')
occupancy_final.head()

,station_id,duration
arrival_time,,
2013-06-27 14:45:00,32,13
2013-06-27 15:05:00,19,1
2013-06-27 15:09:00,19,4
2013-06-27 19:03:00,76,652
2013-06-27 19:51:00,75,754


### Export Clean Data

In [23]:
# export clean data to csv
occupancy_final.to_csv('data_clean/occupancy.csv', index=True, header=True)

In [61]:
del occupancy